In [8]:
intention = '''Create an easy/safe API to parse/hold/yield constructions of pydantic docx class objects.
In theory this can be generalized and abstracted since the classes are largely absracted. 
However since this is API based, I will just focus on solutions for THIS dictionary task. These can be expanded later, or just have other APIs added
Final objective is have a tool to pass complex commands/instructions, and have it yield a pandas dataframe of the results.
Intermediate steps may be yield a list of lists
I am not yet sure if there is benefit trying to have the whole dictionary in one notional object. 
I don't think so, since each operation is psuedo atomic, and any aggregate actions are intended to be done in something like pandas.
Eventually I would want a loop for pandas operations to easily feed updates to the data. 
'''
conclusion = '''create a pular dictionary specifc datastructure to handle the aggregate paragraphs. 
This will only be per lemma, 
as any aggregations from roots and relations of that sort can be managed in pandas and created by simple inheretance/reference
'''

In [42]:
from typing import Optional, Dict, List, Any, Union, Tuple
import pydantic
from pydantic import ValidationError, validator, root_validator, Field, constr
from pydantic_docx import Docx_Paragraph_and_Runs #type:ignore
import re
import json
from itertools import compress, chain
from datetime import datetime

In [10]:
outputs = [
   
   'txt_FulaLemmas',
   'txt_FulaRoots',
   
   #extra
   'txt_FulaAnnotations',
   #gloss derivatives
   'txt_FulaSenseEnglish',
   'txt_FulaSenseFrench',
   'txt_FulaSenseClassifications',
   'txt_FulaSenseEnglishAnnotations',
   'txt_FulaSenseFrenchAnnotations',

   #lemma derivative
   'txt_FulaDialects',
   'txt_FulaPOSTags',
   'txt_FulaSynonyms',


   #agg
   'txt_FulaInParenthesis',
   'txt_POS',
   'txt_RootOrigins'
]

In [11]:
noun_patterns = [r'n\.',r"(?<=\()[^\)]+",r"\/"]
nounPatternRegex = re.compile('|'.join([p for p in noun_patterns]))
print(nounPatternRegex)
s = 'n. dfhjkgqh (wsh) askldjhf / asdfhjkkh'
print(re.findall(nounPatternRegex,s))
print(nounPatternRegex.findall(s))

# if /, the next word is a plural. The plural also then usually has a (el) class after it. commas are multiple version
# so each lemma noun may have a (class). The lemma may have one or multiple plurals, which will have a modified (class)
# so have a nouns list? nouns may then be 
# [('n.',{bool}), #only needed if this gets added to all POS? Or maybe theres information in its use?
# ([[{word}],{Optional[class]}]),  #singular
   #([[{word}],{Optional[class]}])]   #plural

# so nounStruct[0][1] is the flag for noun?
   # one POS table, and noun will get added there. But the presence of a noun indicates to look here
# classes may be found [sClass[1] for sClass in nounStruct[1]] will give all classes of singular, [sClass[1] for sClass in nounStruct[1]]

re.compile('n\\.|(?<=\\()[^\\)]+|\\/')
['n.', 'wsh', '/']
['n.', 'wsh', '/']


In [ ]:
# now = datetime.now()
# current_time = now.strftime("%Y-%m-%d_-_%H-%M-%S")
# print(f"Experiment time: {current_time}\nExperiment note: {experiment}\n\n")

In [25]:
def getAssert(dct,val,typ:type) -> Any:
   out = dct.get(val,False)
   assert isinstance(out,typ), f"the provided dict {dct} did not give {val} with the expected type: {typ}"
   return out

In [1]:


class dict_entry(pydantic.BaseModel): 
   '''
   #True is used as default value, and must be changed to False if the feature is not found. This will catch not implemented, and incomplete updates
   '''
   irregularities: List[str] = []
   lemma_index: int 
   paragraphs: List[Docx_Paragraph_and_Runs] = Field(...,min_items = 1)
   lemma: str = Field(...,min_length = 1) #article
   root: str
   root_subpiece: Union[str,bool] = True #these will not be updated after initialization
   root_metadata: Dict[str,Any]  #index,root_text, root_meta_data #article 
   root_origin: Union[str,bool] = True #these will not be updated after initialization
   #Paragraph Derivatives: Glosses and Annotations
   lemmaLine_runs: List[str] #these will not be updated after initialization
   englishGlossLine_runs: Union[List[str],bool] = True #these will not be updated after initialization
   frenchGlossLine_runs: Union[List[str],bool] = True #these will not be updated after initialization
   FulaAnnotations_runs: Union[List[List[str]],bool]= True #these will not be updated after initialization

   #Gloss Derivatives: English Senses
   FulaSenseEnglish: Union[List[str],bool] = True #list of "senses" split by semicolons #article
   FulaSenseEnglish_Count: Union[int,bool] = True #number of senses in english #aka FulaSenseClassifications #article
   FulaSenseEnglish_Annotations: Union[List[str],bool] = True #contains the annotations (in parenthesis) for a given sense, if any #article 
   FulaSenseEnglish_Synonyms: Union[List[str],bool] = True #holds bracket text for a sense, suspected all synonyms. These may all occur at the end, and may be redundant with the synonyms provided at the head of the entry
   FulaSenseEnglish_unusedContent: Union[List[str],bool] = True #holds any run text that is not contained in the above features
   
   #Gloss Derivatives: French Senses
   FulaSenseFrench: Union[List[str],bool] = True #article
   FulaSenseFrench_Count: Union[int,bool] = True #aka FulaSenseClassifications
   FulaSenseFrench_Annotations: Union[List[str],bool] = True
   FulaSenseFrench_Synonyms: Union[List[str],bool] = True
   FulaSenseFrench_unusedContent: Union[List[str],bool] = True
   
   #lemma derivative
   lemmaLine_unusedContent: List[Tuple[int,str]] #= Field(...) #int is index for run to allow tracing as entries are removed
   FulaDialects: Union[List[str],bool] = True #article
   FulaPOSTags: Union[List[str],bool] = True #article
   FulaPOSClass: Union[List[str],bool] = True #Noun, Adj, Verb, Adv, Prn, ..., Complicated, Indeterminate
   FulaNoun_NounsAndClass: Union[List[Tuple[str,str]],bool] = True #Pular has unique noun classes. Lemma will be copied here beside its class ("noun", "nounclass"), and any additional singular forms will be in additional tuples in this same list
   FulaNoun_PluralsAndClass: Union[List[Tuple[str,str]],bool] = True #Dict provides plurals for nouns. Tuples will have ("noun", "nounclass")
   FulaSynonyms: Union[List[str],bool] = True #article
   FulaCrossRef: Union[List[str],bool] = True
         # FulaVerbClass:
   
   class Config:
      validate_all = True
      validate_assignment = True
      smart_union = True  
      extra = 'forbid'

   @root_validator(pre=True)
   def _validate_and_build(cls, values: Dict[str,Any]) -> Dict[str, Any]:
      # print(values)
      newValues: Dict[str,Any] = {}
      newValues['irregularities'] = []

      ###PARAGRAPHS CHECK###
      newValues['paragraphs'] = getAssert(values,'paragraphs',list)
      if len(newValues['paragraphs']) == 0:
         raise ValueError('given invalid paragraphs of zero length')
      else: #logic for subsidary paragraphs
         lemma_line = newValues['paragraphs'][0].trustyGet(feat = 'run_text', silent_return=False)
         newValues['lemmaLine_runs'] = lemma_line
         if len(newValues['paragraphs']) == 1:
            newValues['englishGlossLine_runs'] = False
            newValues['frenchGlossLine_runs'] = False
         elif len(newValues['paragraphs']) == 2:
            newValues['irregularities'].append('What:ambiguous, Where:paragraphs, Why: only one')
            newValues['englishGlossLine_runs'] = False
            newValues['frenchGlossLine_runs'] = False
         elif len(newValues['paragraphs']) == 3:
            newValues['englishGlossLine_runs'] = newValues['paragraphs'][1].get_run_text()
            newValues['frenchGlossLine_runs'] = newValues['paragraphs'][2].get_run_text()
         else: 
            newValues['englishGlossLine_runs'] = newValues['paragraphs'][1].get_run_text()
            newValues['frenchGlossLine_runs'] = newValues['paragraphs'][2].get_run_text()
            newValues['FulaAnnotations_runs'] = newValues['paragraphs'][3:].get_run_text()
      
      ###LEMMA CHECK###
      #reading in lemma results from first pass of pydantic parser
      lemma_index, lemma_mask, lemmaLine_runs = getAssert(values,'lemma',list)
      #checking for correct structure
      lemma_matched_runs = list(compress(lemmaLine_runs,lemma_mask))
      if len(lemma_matched_runs) > 1:
         newValues['irregularities'].append('What:Unexpected, Where: Lemmas, Why: the merge routines should aggregate adjacent runs with same features. Multiple Lemma runs should not be possible if Bold is contiguous')
      lemma_text = ''.join(chain(lemma_matched_runs)).strip()
         #TODO have better control for expected structure that these runs should be adjacent (and only should be one)
      #saving values. These will have to pass type check of declared attribute types for validation to succeed
      newValues['lemma'] = lemma_text
      newValues['lemma_index'] = lemma_index
      newValues['lemmaLine_runs'] = lemmaLine_runs
      used_run_mask = lemma_mask
      ###ROOT CHECK###
      #reading in lemma results from first pass of pydantic parser
      root_container = [(ind, mask, rootLine_runs) for ind, mask, rootLine_runs in getAssert(values,'root',list)]
      #checking for correct structure
      root_index, root_mask, rootLine_runs = root_container[0]
      root_matched_runs = list(compress(rootLine_runs,root_mask))
      if len(root_matched_runs) > 1:
         newValues['irregularities'].append('What:Unexpected, Where: Root, Why: the merge routines should aggregate adjacent runs with same features. Multiple Root runs should not be possible if Fontsize is contiguous and unique')
      root_text = ''.join(chain(root_matched_runs)).strip()
      if root_index == newValues['lemma_index']:
         newValues['irregularities'].append('What:inconsistent, Where: Lemmas and Root, Why:normally root and lemma are on different lines. This has them sharing, which may indicate a lack of other content')
         used_run_mask = [any(run) for run in list(zip(used_run_mask,root_mask))]
         rootLine_runs = False
      newValues['root'] = root_text
      newValues['root_metadata'] = {'root_index': root_index, 'root_runs':rootLine_runs}
      ###ROOT-Subpiece CHECK###
      #iterating in case a subroot is present
      if len(root_container) == 2:
         root_index, root_mask, rootLine_runs = root_container[1]
         root_matched_runs = list(compress(rootLine_runs,root_mask))
         if len(root_matched_runs) > 1:
            newValues['irregularities'].append('What:Unexpected, Where: Root-Subpiece, Why: the merge routines should aggregate adjacent runs with same features. Multiple Root runs should not be possible if Fontsize is contiguous and unique')
         root_subpiece_text = ''.join(chain(root_matched_runs)).strip()
         if root_index == newValues['lemma_index']:
            newValues['irregularities'].append('What:inconsistent, Where: Lemmas and Root-Subpiece, Why:normally root and lemma are on different lines. This has them sharing, which may indicate a lack of other content')
            used_run_mask = [any(run) for run in list(zip(used_run_mask,root_mask))]
            rootLine_runs = False
         newValues['root_subpiece'] = root_subpiece_text
         newValues['root_metadata'] = {'root_subpiece_index': root_index, 'root_subpiece_runs':rootLine_runs}
      else:
         newValues['root_subpiece'] = False
      #ROOT INPUT INTEGRITY CHECK
      #controlling for unconstrained input root_container list
      if len(root_container) > 2:
         raise NotImplementedError('more than two roots were passed. This is either incorrect or not supported')
      
      #determining lemmaLine_unusedContent
      unused_run_mask = [not r for r in used_run_mask]
      newValues['lemmaLine_unusedContent'] = compress(enumerate(newValues['lemmaLine_runs']),unused_run_mask)
      
      return newValues

   def parse_senses(self):
      #TODO
      return

   def parse_glossRemainder(self):
      #TODO
      return

   def parse_lemmaLine(self):
      #TODO
      return

   def parse_lemmaLineRemainder(self):
      #TODO
      return

   def get_rootOrigin(self):
      #TODO
      return
 
   def give_entryText(self, joiner = '\t') -> str: #article
      return joiner.join([para.trustyGet('para_text') for para in self.paragraphs])

   def trustyGet(self, feat: str, silent_return = True) -> str:
      if not silent_return:
         output : str = getattr(self,feat,'')
         if len(output) > 0:
            return output
         else:
            raise AttributeError(f'trusty getter could not find: {feat}')
      else:
         output : str = getattr(self,feat,'')
         return output
      
            
dict_entry.parse_obj({'paragraphs':[b for a,b in parsed_object_list[5:8]]})

# incoming = {
#    'paragraphs': List[Docx_Paragraph_and_Runs],
#    'root': List[int,str,List[bool]],
#    'lemma': List[int,str,List[bool]],
#    # 'pos': 
# }

# [
#    ['paras'], #obj
#    ['parsed_paras'], #pydocx
#    ['roots'], #name, fill forwards
#    ['lemmas']  #text, fill forwards
# ]

SyntaxError: invalid syntax (915979843.py, line 77)

In [27]:
import pickle
with open('parsed_objectClass_outcomes_dict.pkl', 'rb') as file:
    # Call load method to deserialze
    parsed_objectClass_outcomes_dict = pickle.load(file, encoding='utf-8')

parsed_object_list = parsed_objectClass_outcomes_dict['parsed_object_list'] 
para_text_lookup = parsed_objectClass_outcomes_dict['para_text_lookup'] 
root_ind_list = parsed_objectClass_outcomes_dict['root_ind_list'] 
subroot_ind_list = parsed_objectClass_outcomes_dict['subroot_ind_list'] 
lemma_ind_list = parsed_objectClass_outcomes_dict['lemma_ind_list'] 
root_and_lemma_one_line = parsed_objectClass_outcomes_dict['root_and_lemma_one_line'] 
root_lookup = parsed_objectClass_outcomes_dict['root_lookup'] 
lemma_lookup = parsed_objectClass_outcomes_dict['lemma_lookup'] 
# char_counts = parsed_objectClass_outcomes_dict['char_counts']

In [1]:
# result = []
# for i in parsed_object_list[5:15]:
#    result.append(dict_entry({i))
# dict_entry.parse_obj({'paragraphs':parsed_object_list[5:8]})

dct = {
   'paragraphs':parsed_object_list[5:8],
   'root': ()
}
ind, mask, rootLine_runs)


NameError: name 'dict_entry' is not defined